Student: Irakli Kelbakiani Email: Irakli.Kelbakiani@students.unibe.ch Github: https://github.com/KelbakianiIrakli/Image-Processing/tree/main/Exercise-2

(a) Define a universal color table with a maximum of 256 different colors. I define 216 colors in a following way: for r in range(0, 256, 51): # Red channel values (0, 51, 102, 153, 204, 255) for g in range(0, 256, 51): # Green channel (0, 51, 102, 153, 204, 255) for b in range(0, 256, 51): # Blue channel (0, 51, 102, 153, 204, 255) color_table.append((r, g, b))

Each of the three nested loops iterates over the red, green, and blue channels, respectively, with step increments of 51. Given this structure, we can calculate the total number of colors generated as follows: For the red channel (r), there are (256 - 0) / 51 + 1 possible values, which is 6 values (0, 51, 102, 153, 204, 255). For the green channel (g), there are also 6 possible values. For the blue channel (b), there are 6 possible values. So, as 666=216 I have total 216 colors defined.

(b) Write an algorithm that transforms the initial pixel values with an index to the color table so that the return image looks as similar as possible to the original image. def find_closest_color_index(pixel, color_table): min_distance = float("inf") closest_index = 0

for i, color in enumerate(color_table):
    # Calculate Euclidean distance between pixel and color
    distance = sum((a - b) ** 2 for a, b in zip(pixel, color)) ** 0.5

    if distance < min_distance:
        min_distance = distance
        closest_index = i

return closest_index
I created a function, in which we calculate Euclidean distance of this pixel and each color in color table. We will take value from the color map which has smallest distance.

(c) Replace the universal color table by an adaptive colour table, which is optimized for the given input image. generate_adaptive_color_table(image, num_colors)- function generates an adaptive color table using K-Means clustering by extracting representative 256 colors from lena image.

(d) Apply your two algorithms on the image ”Lena” which is on ILIAS. 
Please, see images in the same dir as this file.

In [20]:
from PIL import Image
lena_image = Image.open("lena.png")

# Generate universal color table - 216 colors in total

In [21]:
def generate_universal_color_table():
    color_table = []

    for r in range(0, 256, 51):  # Red channel values (0, 51, 102, 153, 204, 255)
        for g in range(0, 256, 51):  # Green channel (0, 51, 102, 153, 204, 255)
            for b in range(0, 256, 51):  # Blue channel (0, 51, 102, 153, 204, 255)
                color_table.append((r, g, b))

    return color_table

# Generate the universal color table
universal_color_table = generate_universal_color_table()

# make a png image for created colors

In [22]:
from PIL import Image, ImageDraw, ImageFont

# color table image for better visualization
def create_color_table_image(color_table, grid_size, cell_size):

    image_width = grid_size[0] * cell_size
    image_height = grid_size[1] * cell_size


    image = Image.new("RGB", (image_width, image_height), (255, 255, 255))
    draw = ImageDraw.Draw(image)


    font = ImageFont.load_default()
    font_size = 9 
    font = ImageFont.truetype("arial.ttf", font_size) 
    for i, color in enumerate(color_table):
        row = i // grid_size[0]
        col = i % grid_size[0]
        x = col * cell_size
        y = row * cell_size
        color_tuple = (color[0], color[1], color[2])
        draw.rectangle([x, y, x + cell_size, y + cell_size], fill=color_tuple)
        text_x = x + 2
        text_y = y + 2
        # use complementary color for each cell so that text is visible
        text_color = (255 - color[0], 255 - color[1], 255 - color[2]) 
        draw.text((text_x, text_y), f"{color}", font=font, fill=text_color)

    return image


grid_size = (16, 16)  
cell_size = 70

# Generate the universal color table
universal_color_table = generate_universal_color_table()

# Create an image of the color table
color_table_image = create_color_table_image(universal_color_table, grid_size, cell_size)

color_table_image.save("universal_color_table.png")

# Find closest color index using Euclidean distance

In [23]:
def find_closest_color_index(pixel, color_table):
    min_distance = float("inf")
    closest_index = 0

    for i, color in enumerate(color_table):
        # Calculate Euclidean distance between pixel and color
        distance = sum((a - b) ** 2 for a, b in zip(pixel, color)) ** 0.5

        if distance < min_distance:
            min_distance = distance
            closest_index = i

    return closest_index

# Generate lena using universal color table

In [24]:
def quantize_to_color_table(image, color_table):
    quantized_image = Image.new("P", image.size)
    
    palette_bytes = bytes([value for color in color_table for value in color])
    
    quantized_image.putpalette(palette_bytes)
    
    for x in range(image.width):
        for y in range(image.height):
            pixel = image.getpixel((x, y))
            index = find_closest_color_index(pixel, color_table)
            quantized_image.putpixel((x, y), index)

    return quantized_image

universal_color_table_bytes = bytes([value for color in universal_color_table for value in color])

quantized_lena = quantize_to_color_table(lena_image, universal_color_table)
quantized_lena.putpalette(universal_color_table_bytes)  
quantized_lena.save("lena_universal_color_table.png")



# Generate adaptive color table using K-Means clustering.

In [25]:
# We use K-Means to identify suitable color table for this image
# After training, the cluster centers represent the representative colors that K-Means has identified. 
# These cluster centers are extracted as the adaptive color table.

In [26]:
from sklearn.cluster import KMeans
import numpy as np

def generate_adaptive_color_table(image, num_colors):
    pixel_data = np.array(image).reshape(-1, 3)

    # Apply K-Means clustering to find representative colors
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(pixel_data)

    adaptive_color_table = kmeans.cluster_centers_.astype(int)

    return adaptive_color_table

adaptive_color_table = generate_adaptive_color_table(lena_image, 256)
grid_size = (16, 16)  
cell_size = 70
color_table_image = create_color_table_image(adaptive_color_table, grid_size, cell_size)
color_table_image.save("adaptive_color_table.png")
quantized_lena_adaptive = quantize_to_color_table(lena_image, adaptive_color_table)
quantized_lena_adaptive.save("lena_adaptive_color_table.png")